# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE'

In [1]:
# Import dependencies
import json
from pymongo import MongoClient
from pprint import pprint


In [2]:
# MongoDB server
client = MongoClient("localhost", 27017)
databases = client.list_database_names()

# Create or access the "uk_food" database
db = client["uk_food"]

# Create or access the "establishments" collection
collection = db["establishments"]

# Load the data from the "establishments.json" file
with open("Resources/establishments.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Insert the data into the "establishments" collection
collection.insert_many(data)


In [3]:
# confirm that our new database was created
if "uk_food" in databases:
    print("The 'uk_food' database was created!")
else:
    print("The 'uk_food' database was not found.")
    

The 'uk_food' database was created!


In [4]:
# assign the uk_food database to a variable name
db = client["uk_food"]

In [5]:
# review the collections in our new database
# Access the "uk_food" database
db = client["uk_food"]

# List all collections in the "uk_food" database
collections = db.list_collection_names()

# Print the list of collections
print(collections)


['establishments']


In [6]:
# review a document in the establishments collection
# Access the "uk_food" database
db = client["uk_food"]

# Access the "establishments" collection
collection = db["establishments"]

# Find one document in the "establishments" collection
document = collection.find_one()

# Display the document using pprint
pprint(document)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64929417af6298e20780d31c'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

In [7]:
# assign the collection to a variable
collection = db["establishments"]

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
        "BusinessName": "Penang Flavours"
}

In [9]:
# Insert the new restaurant into the collection
collection = db["establishments"]
result = collection.insert_one(new_restaurant)


In [10]:
# Check that the new restaurant was inserted
document = collection.find_one({"BusinessName": "Penang Flavours"})

# Check if a document was found
if document:
    print("The new restaurant was inserted!")
else:
    print("The new restaurant was not found.")


The new restaurant was inserted!


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
document = collection.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}
)

# Check if a document was found
if document:
    # Print the BusinessTypeID and BusinessType values
    print(f"BusinessTypeID: {document['BusinessTypeID']}")
    print(f"BusinessType: {document['BusinessType']}")
else:
    print("No document was found.")
    

BusinessTypeID: 1
BusinessType: Restaurant/Cafe/Canteen


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
business_type_id = 12345

result = collection.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type_id}}
)

# Print the result of the update operation
print(f"Matched: {result.matched_count}")
print(f"Modified: {result.modified_count}")

Matched: 1
Modified: 0


In [13]:
# Confirm that the new restaurant was updated
if document:
    # Print the BusinessTypeID value of the new restaurant
    print(f"BusinessTypeID: {document['BusinessTypeID']}")
else:
    print("The new restaurant was not found.")
    

BusinessTypeID: 1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
count = collection.count_documents({"LocalAuthorityName": "Dover"})

# Print the results
print(f"Number of documents with LocalAuthorityName set to 'Dover': {count}")


Number of documents with LocalAuthorityName set to 'Dover': 994


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
result = collection.delete_many({"LocalAuthorityName": "Dover"})

# Print the results
print(f"Deleted: {result.deleted_count}")


Deleted: 994


In [16]:
# Check if any remaining documents include Dover
count = collection.count_documents({"LocalAuthorityName": "Dover"})

# Check if any documents were found
if count > 0:
    print(f"There are {count} remaining documents with LocalAuthorityName set to 'Dover'.")
else:
    print("There are no remaining documents with LocalAuthorityName set to 'Dover'.")


There are no remaining documents with LocalAuthorityName set to 'Dover'.


In [17]:
# Check that other documents remain with 'find_one'
document = collection.find_one()

# Check if a document was found
if document:
    print("There are remaining documents in the 'establishments' collection.")
else:
    print("There are no remaining documents in the 'establishments' collection.")


There are remaining documents in the 'establishments' collection.


5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude
documents = collection.find({"Longitude": {"$type": "string"}})

# Iterate over the documents
for document in documents:
    # Convert Latitude to a decimal number
    longitude = float(document["Longitude"])

    # Update the document with the converted value
    collection.update_one(
        {"_id": document["_id"]},
        {"$set": {"Longitude": longitude}}
    )


In [19]:
# Change the data type from String to Decimal for latitude
documents = collection.find({"Latitude": {"$type": "string"}})

# Iterate over the documents
for document in documents:
    # Convert Latitude to a decimal number
    latitude = float(document["Latitude"])

    # Update the document with the converted value
    collection.update_one(
        {"_id": document["_id"]},
        {"$set": {"Latitude": latitude}}
    )


In [20]:
# Check that the coordinates are now numbers
# Find documents where Longitude is still a string
longitude_count = collection.count_documents({"Longitude": {"$type": "string"}})

# Find documents where Latitude is still a string
latitude_count = collection.count_documents({"Latitude": {"$type": "string"}})

# Print the results
print(f"Number of documents with Longitude as a string: {longitude_count}")
print(f"Number of documents with Latitude as a string: {latitude_count}")


Number of documents with Longitude as a string: 0
Number of documents with Latitude as a string: 0
